# Query Store - Queries that run longer than 30 Seconds

In [ ]:
SELECT TOP (100)
    p.query_id query_id,
	p.plan_id plan_id,
    q.object_id,
	ISNULL(OBJECT_NAME(q.object_id),'Unknown') object_name,
    qt.query_sql_text query_sql_text,
	ROUND(CONVERT(float, SUM(rs.max_duration*rs.count_executions))/NULLIF(SUM(rs.count_executions), 0)*0.001,2) max_duration_ms,
	MAX(rs.last_execution_time) AS 'LastExecutiontime',
    SUM(rs.count_executions) count_executions,
    COUNT(DISTINCT p.plan_id) num_plans
FROM sys.query_store_runtime_stats rs
    JOIN sys.query_store_plan p ON p.plan_id = rs.plan_id
    JOIN sys.query_store_query q ON q.query_id = p.query_id
    JOIN sys.query_store_query_text qt ON q.query_text_id = qt.query_text_id
WHERE 
	(rs.last_execution_time >= '01-01-2022' AND rs.last_execution_time <= getdate())
GROUP BY 
	p.query_id, p.plan_id, qt.query_sql_text, q.object_id
-- 15 Second Timeouts
HAVING 
	ROUND(CONVERT(float, SUM(rs.max_duration*rs.count_executions))/NULLIF(SUM(rs.count_executions), 0)*0.001,2)>15000
ORDER BY 
	6 DESC


# Whats Running Now

In [ ]:
--- 1 Work In Process
--- Ignore me - This is not the query you are looking for
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
USE master
GO

SELECT
	sp.session_Id AS 'Spid'
	,er.blocking_session_id AS 'BlockedBy'
	,DB_NAME(er.database_id) AS 'Database'
	,Host_name AS 'HostName'
	,sp.login_name AS 'User'
	,er.command AS 'Command'
	,CASE er.transaction_isolation_level
		WHEN 1 THEN 'ReadUncommitted'
		WHEN 2 THEN 'ReadCommitted'
		WHEN 3 THEN 'Repeatable'
		WHEN 4 THEN 'Serializable'
		WHEN 5 THEN 'Snapshot'
		ELSE 'Unspecified'
	END	AS TrnIsoLevel
	,er.dop AS 'Dop'
	,sp.open_transaction_count AS 'OpenTrans'
	,er.cpu_time AS 'CPUTime'
	,er.Logical_reads AS 'Logical Reads'
	,er.reads
	,er.writes
	,er.row_count AS 'Rows'
	,er.granted_query_memory as 'MemGrantKB'
	,er.status AS [Status]
	,er.wait_type AS [WaitType]
	,program_name AS 'Application'
	,p.query_plan as 'QueryPlan'
	,CAST('<?query --'+CHAR(13)+SUBSTRING(qt.text,
	(er.statement_start_offset / 2)+1,
	((CASE er.statement_end_offset
		WHEN -1 THEN DATALENGTH(qt.text)
		ELSE er.statement_end_offset
	  END - er.statement_start_offset)/2) + 1)+CHAR(13)+'--?>' AS XML) AS SqlStmt
	, qt.text AS [ParentQuery]
	,er.request_id AS 'RequestID'
	,er.percent_complete
	,start_time AS 'Started'
	,DATEADD(ms,er.estimated_completion_time,GETDATE()) AS [ETA Completion]
	,CONVERT(NUMERIC(10,2),er.estimated_completion_time/1000.0/60.0) AS [ETA Mins]
FROM
	sys.dm_exec_requests er
INNER JOIN
	sys.dm_exec_sessions sp
ON
	er.session_id = sp.session_id
OUTER APPLY
	sys.dm_exec_sql_text(er.sql_handle)AS qt
OUTER APPLY
	sys.dm_exec_query_plan(er.plan_handle) p
WHERE
	sp.is_user_process = 1
	/* sp.session_Id > 50
	-- Ignore system spids. -- */
	AND sp.session_Id <> @@SPID
	-- Search for Specific Query Text
	-- AND qt.text like N'%'+N'vStatement'+N'%'
	--ORDER BY 
	--1, 2
	--cpu_time desc
	AND	sp.program_name NOT LIKE 'DatabaseMail%'

--- 2 Locking
--- Look for any Locks blocking others 
SELECT
	*
FROM
	sys.dm_os_waiting_tasks t
inner join
	sys.dm_exec_connections c
ON
	c.session_id = t.blocking_session_id
cross apply
	sys.dm_exec_sql_text(c.most_recent_sql_handle) as h1

--- 3 Current Sessions
exec sp_who2

--- 4 show all threads, use Process Explorer to suspend the thread KPID
--select * from master..sysprocesses --where spid = 64


# Current Locks

In [ ]:
SELECT 
    HostName,
    "OS UserName",
    Login, 
    spid, 
    "Database", 
    TableID,
    "Table Name", 
    IndID,   
    "Lock Type", 
    "Lock Mode", 
	LM,
    Status, 
  Resource,
    COUNT(*) AS "Lock Count"
FROM (
    SELECT
        CONVERT(VARCHAR(30), RTRIM(P.HostName)) AS 'HostName',
        CONVERT(VARCHAR(30), RTRIM(P.nt_UserName)) AS 'OS UserName',
        CONVERT(VARCHAR(30), SUSER_SNAME(p.sid)) AS 'Login', 
        CONVERT(SMALLINT,req_spid) AS 'spid',
        CONVERT(VARCHAR(30), DB_NAME(rsc_dbid)) AS 'Database',
        rsc_objid AS 'TableID',
		CONVERT(VARCHAR(30), OBJECT_NAME(rsc_objid, rsc_dbid)) AS 'Table Name',
        rsc_indid AS 'IndID', 
        CASE SUBSTRING (lock_type.name, 1, 4) 
            WHEN '' THEN 'None'
            WHEN 'DB' THEN 'Database'
            WHEN 'FIL' THEN 'File'
            WHEN 'IDX' THEN 'Index'
            WHEN 'TAB' THEN 'Table'
            WHEN 'PAG' THEN 'Page'
            WHEN 'KEY' THEN 'Key'
            WHEN 'EXT' THEN 'Extent'
            WHEN 'RID' THEN 'Row ID'
            WHEN 'APP' Then 'Application'
            Else SubString (lock_type.name, 1, 4)
        END AS 'Lock Type',		
        Case SubString (lock_mode.name, 1, 12)
            When NULL Then 'N/A'
            When 'Sch-S' Then 'SCHEMA (Stability)'--'SCHEMA stability lock'
            When 'Sch-M' Then 'SCHEMA (Modification)'--'SCHEMA modification lock'
            When 'S' Then 'SHARED'--'SHARED Lock acquisition'
            When 'U' Then 'UPDATE'--'UPDATE lock acquisition'
            When 'X' Then 'EXCLUSIVE'--'EXCLUSIVE lock granted'
            When 'IS' Then 'SHARED (Intent)'--'INTENT for SHARED lock'
            When 'IU' Then 'UPDATE (Intent)'--'INTENT for UPDATE lock'
            When 'IX' Then 'EXCLUSIVE (Intent)'--'INTENT for EXCLUSIVE lock'
            When 'SIU' Then 'SHARED (Intent UPDATE)'--'SHARED lock with INTENT for UPDATE'
            When 'SIX' Then 'SHARED (Intent EXCLUSIVE)'--'SHARED lock with INTENT for EXCLUSIVE'
            When 'UIX' Then 'UPDATE'--'UPDATE lock with INTENT for EXCLUSIVE'
            When 'BU' Then 'UPDATE (BULK)'--'BULK UPDATE lock'
            Else SubString (lock_mode.name, 1, 12)
        END AS 'Lock Mode', 
		SubString (lock_mode.name, 1, 12) AS 'LM',
        SubString(lock_status.name, 1, 5) AS 'Status',
        SubString (rsc_text, 1, 16) AS 'Resource'
    FROM 
        Master..SysLockInfo S
        JOIN Master..spt_values lock_type on S.rsc_type = lock_type.number
        JOIN Master..spt_values lock_status on S.req_status = lock_status.number
        JOIN Master..spt_values lock_mode on S.req_mode = lock_mode.number -1
        JOIN Master..SysProcesses P on S.req_spid = P.spid
    WHERE
            lock_type.type = 'LR'
        AND lock_status.type = 'LS'
        AND lock_mode.type = 'L'
        AND DB_Name(rsc_dbid) NOT IN ('master', 'msdb', 'model')
    ) AS X
WHERE TableID > 0
GROUP BY 
    [HostName],
    [OS UserName],
    [Login], 
    [spid], 
    [Database], 
    [TableID],
    [Table Name], 
    [IndID], 
    [Lock Type], 
    [Lock Mode], 
	[LM],
    [Status],
	resource
ORDER BY
    [spid], [Database], [Table Name], [Lock Type], [Login]

--- Check the Messages Tab
DBCC opentran

# Index Performance

In [ ]:
--- Index Scan/Seek/Lookup/Update Breakdown

SELECT
	DB_NAME([ddius].[database_id]) AS [database name],
	s.[name] AS 'Schema_Name',
	OBJECT_NAME([ddius].[object_id]) AS [Table name],
	CASE
		WHEN ddius.index_id=1 THEN '*'+[i].[name]+'*'
		ELSE [i].[name]
	END AS [index name],
	i.is_unique,	
	ddius.database_id,
	--ddius.object_id,
	ddius.index_id,
	p.partition_number,
	ddius.user_seeks,
	ddius.user_scans,
	ddius.user_lookups,
	ddius.user_updates,
	ddius.system_seeks,
	ddius.system_scans,
	ddius.system_lookups,
	ddius.system_updates,
	CAST(p.used_page_count * 0.0078125 AS NUMERIC(18,2)) AS 'UsedPages_MB',
	p.[ROW_COUNT] AS 'RowCount'
FROM
	[sys].[dm_db_index_usage_stats] AS ddius
INNER JOIN
	[sys].[indexes] AS i
ON
	[ddius].[index_id] = [i].[index_id] AND [ddius].[object_id] = [i].[object_id]
JOIN
	sys.dm_db_partition_stats p
ON	
	i.OBJECT_ID = p.OBJECT_ID AND i.index_id = p.index_id
JOIN
	sys.objects O
ON O.object_id = ddius.object_id
JOIN
	sys.schemas S
ON S.schema_id = O.schema_id
WHERE
	OBJECT_NAME([ddius].[object_id])<>'sysdiagrams' -- filter out sysdiagrams UML table
    AND OBJECTPROPERTY(ddius.OBJECT_ID, 'IsUserTable') = 1 -- User Tables Only
	AND ddius.index_id > 0  -- filter out heaps
	AND ddius.database_id = db_id() -- current DB only
	AND ddius.index_id>1 -- NCIX only
ORDER BY
	1,2,3

# Connections Per Database

In [ ]:
--- Simple
SELECT
	@@SERVERNAME AS 'Server',
	DB_NAME(dbid) as 'Database',
	COUNT(dbid) as 'Number Of Open Connections'
FROM
	sys.sysprocesses
WHERE
	dbid > 0
GROUP BY
	dbid
with rollup
order by
	2

--- Detailed
SELECT
	@@SERVERNAME AS 'Server',
	sd.name DBName,
	sp.dbid,
    loginame [Login],
    hostname,
	[program_name] 'Program Name',
	[spid],
    last_batch LastBatch,
    blocked BlkBy,
	sp.[status],
	cmd Command,
    cpu CPUTime,
    physical_io DiskIO
FROM sysprocesses sp 
JOIN sysdatabases sd ON sp.dbid = sd.dbid
where sp.loginame<>'sa'
ORDER BY 2, 4